In [ ]:
import numpy as np 
from nltk.tokenize import word_tokenize
from src.MPNeuronInfo import Loss_CrossCategoricalEntropy, OptimizerAdam
from src.transformer import Encoder, Decoder

In [ ]:
import sys
sys.path.append("..")


In [ ]:
import nltk
nltk.download('punkt_tab')

Initialisation

In [ ]:

corpus = ["""
The cat is sleeping on the bed while the dog is barking loudly near the street full of cars. 
The sun is shining brightly above the sky, and the bird is flying high near the mountains. 
The child is running in the park, and the flower is blooming beautifully in the garden. 
The man is eating an apple while the woman is reading a book. 
The boy is playing football and the girl is singing a song happily. 
The apple is red and sweet, the sky is blue today, and the water is cold and fresh. 
The food is tasty and hot, and the street is busy with cars and people waiting in line. 
The teacher is writing on the board, and the students are studying hard. 
The baby is crying loudly while the family is having dinner together and the friends are talking happily. 
The dog is sleeping under the tree while the cat is drinking milk. 
The birds are building a nest and the sun sets behind the mountains. 
The stars are shining at night, the wind is blowing softly, and the rain is falling on the roof. 
The car is moving fast, the train is arriving at the station, and the airplane is flying high in the sky. 
The boy is eating ice cream, the girl is watching television, the man is driving a car, and the woman is cooking food. 
The children are playing together while the phone is ringing loudly, the clock is ticking slowly, the lights are glowing brightly, and the music is playing softly.
"""
]

corpus_sp = ["""El gato está durmiendo en la cama mientras el perro está ladrando fuerte cerca de la calle llena de coches. 
El sol está brillando intensamente sobre el cielo, y el pájaro está volando alto cerca de las montañas. 
El niño está corriendo en el parque, y la flor está floreciendo hermosamente en el jardín. 
El hombre está comiendo una manzana mientras la mujer está leyendo un libro. 
El chico está jugando al fútbol y la niña está cantando una canción felizmente. 
La manzana es roja y dulce, el cielo está azul hoy, y el agua está fría y fresca. 
La comida está sabrosa y caliente, y la calle está ocupada con coches y personas esperando en la fila. 
El maestro está escribiendo en la pizarra mientras los estudiantes están estudiando mucho. 
El bebé está llorando fuerte mientras la familia está cenando junta y los amigos están hablando felices. 
El perro está durmiendo bajo el árbol mientras el gato está bebiendo leche. 
Los pájaros están construyendo un nido y el sol se pone detrás de las montañas. 
Las estrellas están brillando por la noche, el viento está soplando suavemente, y la lluvia está cayendo sobre el techo. 
El coche se mueve rápido, el tren está llegando a la estación, y el avión está volando alto en el cielo. 
El chico está comiendo helado, la niña está viendo televisión, el hombre está conduciendo un coche, y la mujer está cocinando comida. 
Los niños están jugando juntos mientras el teléfono está sonando fuerte, el reloj está marcando lentamente, las luces están brillando intensamente, y la música está sonando suavemente."""]



corpus[0] += ' eos'
corpus_sp[0] += ' eos'

e = Encoder(corpus, 4)
d = Decoder(corpus_sp, 4)

e_embd = []

input = [
    "the cat is sleeping",
    "the dog is barking",
    "the sun is shining",
    "the bird is flying",
    "the child is running",
]
    # "the flower is blooming",
    # "the man is eating",
    # "the woman is reading",
    # "the boy is playing",
    # "the girl is singing",
    # "the apple is red",
    # "the sky is blue",
    # "the water is cold",
    # "the food is tasty",
    # "the street is busy"
tokenized_input = [word_tokenize(sentence.lower()) for sentence in input]
n = len(tokenized_input[0])

decoder_input = [
    "el gato está durmiendo",
    "el perro está ladrando",
    "el sol está brillando",
    "el pájaro está volando",
    "el niño está corriendo",

]

    # "la flor está floreciendo",
    # "el hombre está comiendo",
    # "la mujer está leyendo",
    # "el chico está jugando",
    # "la niña está cantando",
    # "la manzana es roja",
    # "el cielo es azul",
    # "el agua está fría",
    # "la comida es sabrosa",
    # "la calle está ocupada"

d_tokenized_input = [word_tokenize(sentence.lower()) for sentence in decoder_input]
m = len(d_tokenized_input[0])

e.vocab_creation()
print("Vocab created for English")
e.word_embeddings()
e.positional_encoding(n+2)
e.query_key_value()
e.feed_forward()
e.ed_key_value()

d.vocab_creation()
print("Vocab created for Spanish \n")
d.word_embeddings()
d.positional_encoding(m+1)
d.query_key_value()
d.feed_forward()
d.ed_query()
d.next_word()

e.map(tokenized_input)
d.map(d_tokenized_input)
d.casual_mask(m+1)
d.label(d_tokenized_input)

loss_fn = Loss_CrossCategoricalEntropy()
y_train = d.one_hot(np.array(d.label), len(d.vocab)-1)

print("Mapped Encoder input : \n", e.input,"\n")
print("Mapped Decoder input : \n", d.input,"\n")

print("Mask created for Decoder - Masked Self Attention : \n",d.mask)

In [ ]:
max_len = len(d_tokenized_input[0])
warmup_steps = 4000
f=0
lrate = 0.00001
for i in range(0, 500):
    result = []
    count = 0
    while count != max_len :
        count += 1
        steps = count + max_len * i
        # lrate = d.embd_dim ** -0.5 * min(steps ** -0.5, steps * warmup_steps ** -1.5)
        
        # lrate = custom_lr(steps, total_steps=2000, warmup_steps=100, base_lr=2e-3, min_lr=1e-5)
        # lrate = initial_lr * (decay_rate ** (steps / decay_steps))

        optimizer = OptimizerAdam(learning_rate=lrate)

        ewe = e.embeddings[(np.array(e.input)).tolist()]
        ewe += e.position_encodings

        e.self_attention(ewe, n)
        e.res = e.layer_normalization(e.res)

        e.ffn1.forward(e.res)
        e.ffnact.forward(e.ffn1.output)
        e.ffn2.forward(e.ffnact.output)

        e.res += e.ffn2.output
        e.res = e.layer_normalization(e.res)
        
        dwe = d.embeddings[(np.array(d.input)).tolist()]
        dwe += d.position_encodings

        d.self_attention(dwe, m)
        d.res = d.layer_normalization(d.res)

        d.ed_attention(e)
        d.final = d.layer_normalization(d.final)

        d.nxtlayer.forward(d.final)
        d.nxtactivation.forward(d.nxtlayer.output)
        d.nxtlayer2.forward(d.nxtactivation.output)
        d.nxtlayer2.output += d.final
        d.nxtlayer2.output = d.layer_normalization(d.nxtlayer2.output)
        
        d.vocablayer.forward(d.nxtlayer2.output)
        d.activation3.forward(d.vocablayer.output)

        check = np.argmax(d.activation3.output, axis=2)

        # Predicted IDs
        # print(check)

        # Calculate loss 
        loss_fn.backward(d.activation3.output, np.array(y_train))

        d.activation3.backward((loss_fn.dinputs))
        d.vocablayer.backward(d.activation3.dinputs)
        d.nxtlayer2.backward(d.vocablayer.dinputs)
        d.nxtactivation.backward(d.nxtlayer2.dinputs)
        d.nxtlayer.backward(d.nxtactivation.dinputs)
       
       
        diff = loss_fn.calculate(d.activation3.output, np.array(y_train))
        print("Loss : ",diff)

        # encoder decoder attention layer 
        ed_v_new_dvals = np.matmul(d.activation2.output.transpose(0,2,1), d.nxtlayer.dinputs)
        e.ed_vlayer.backward(ed_v_new_dvals)

        ed_softmax_dvals = np.matmul(d.nxtlayer.dinputs, e.ed_vlayer.output.transpose(0,2,1))
        d.activation2.backward(ed_softmax_dvals)

        ed_q_new_dvals = np.matmul(d.activation2.dinputs, e.ed_klayer.output)
        d.ed_qlayer.backward(ed_q_new_dvals)
        
        ed_k_new_dvals = np.matmul(d.activation2.dinputs.transpose(0,2,1), d.ed_qlayer.output)
        e.ed_klayer.backward(ed_k_new_dvals)

        # decoder attention layer
        d_v_new_dvals = np.matmul(d.activation.output.transpose(0,2,1), d.ed_qlayer.dinputs)
        d.vlayer.backward(d_v_new_dvals)

        d_softmax_dvals = np.matmul(d.ed_qlayer.dinputs, d.vlayer.output.transpose(0,2,1))
        d.activation.backward(d_softmax_dvals)

        d_q_new_dvals = np.matmul(d.activation.dinputs, d.klayer.output)
        d.qlayer.backward(d_q_new_dvals)

        d_k_new_dvals = np.matmul(d.activation.dinputs, d.qlayer.output)
        d.klayer.backward(d_k_new_dvals)

        # encoder attention layer
        encoder_dinputs = e.ed_vlayer.dinputs + e.ed_klayer.dinputs

        e.ffn2.backward(encoder_dinputs)
        e.ffnact.backward(e.ffn2.dinputs)
        e.ffn1.backward(e.ffnact.dinputs)

        e_v_new_dvals = np.matmul(e.activation.output.transpose(0,2,1), encoder_dinputs)
        e.vlayer.backward(e_v_new_dvals)

        e_softmax_dvals = np.matmul(encoder_dinputs, e.vlayer.output.transpose(0,2,1))
        e.activation.backward(e_softmax_dvals)

        e_q_new_dvals = np.matmul(e.activation.dinputs, e.klayer.output)
        e.qlayer.backward(e_q_new_dvals)
        
        e_k_new_dvals = np.matmul(e.activation.dinputs, e.qlayer.output)
        e.klayer.backward(e_k_new_dvals) 
        
        optimizer.update_params(d.vocablayer)
        optimizer.update_params(d.nxtlayer2)
        optimizer.update_params(d.nxtlayer)

        optimizer.update_params(e.ed_vlayer)
        optimizer.update_params(e.ed_klayer)
        optimizer.update_params(d.ed_qlayer)

        optimizer.update_params(d.vlayer)
        optimizer.update_params(d.klayer)
        optimizer.update_params(d.qlayer)
        
        optimizer.update_params(e.ffn2)
        optimizer.update_params(e.ffn1)

        optimizer.update_params(e.vlayer)
        optimizer.update_params(e.klayer)
        optimizer.update_params(e.qlayer)



In [ ]:
input = ["the cat is sleeping"]
d_input = []
tokenized_input = [word_tokenize(sentence.lower()) for sentence in input]
d_tki = [[0]]

n = len(tokenized_input[0])
e.positional_encoding(n)
e.map(tokenized_input)
e.input = [e.input[0][1:-1]]
print(e.input)
ewe = e.embeddings[(np.array(e.input)).tolist()]
e.self_attention(ewe, n)
e.res = e.layer_normalization(e.res)
e.ffn1.forward(e.res)
e.ffnact.forward(e.ffn1.output)
e.ffn2.forward(e.ffnact.output)

e.res += e.ffn2.output
e.res = e.layer_normalization(e.res)
flag = 1

for _ in range(n):
    # d.map(d_tki)
    m = len(d_tki)
    d.positional_encoding(m)
    d.casual_mask(m)

    dwe = d.embeddings[(np.array(d_tki)).tolist()]
    dwe += d.position_encodings

    d.self_attention(dwe, m)
    d.res = d.layer_normalization(d.res)

    print("e.res shape:", e.res.shape)          
    print("d.final shape before ed_attention:", d.res.shape) 

    d.ed_attention(e)
    d.final = d.layer_normalization(d.final)

    d.nxtlayer.forward(d.final)
    d.nxtactivation.forward(d.nxtlayer.output)
    d.nxtlayer2.forward(d.nxtactivation.output)
    d.nxtlayer2.output += d.final
    d.nxtlayer2.output = d.layer_normalization(d.nxtlayer2.output)
    
    d.vocablayer.forward(d.nxtlayer2.output)
    d.activation3.forward(d.vocablayer.output)

    probs_last = d.activation3.output[0, -1]   
    
    topk = 10
    idxs = np.argsort(probs_last)[-topk:][::-1]
    print("top probs:", [(int(i), float(probs_last[i])) for i in idxs])
    
    check = int(np.argmax(probs_last))
    print(check)

    d_tki[0].append(check)
    print(d_tki)

    if check == 126:
        break



In [ ]:
# assume gold target tokens from training for "the cat is sleeping" were [1,2,3,4,126] (your print)
gold = [1,2,3,4,126]
d_tki = [[0]]  # sos
for t in range(len(gold)):
    # feed decoder with ground-truth tokens up to t (teacher forcing)
    seq = [0] + gold[:t]   # 0 is sos
    d_tki = [seq]
    # do decoder forward exactly as in inference
    dwe = d.embeddings[np.array(d_tki).tolist()]
    dwe += d.position_encodings[:len(seq)]
    d.self_attention(dwe, len(seq))
    d.ed_attention(e)
    d.nxtlayer.forward(d.final)  # etc - run through to logits
    d.vocablayer.forward(d.nxtlayer2.output)
    d.activation3.forward(d.vocablayer.output)
    # check last-step argmax
    probs_last = d.activation3.output[0, -1]
    print("step", t, "pred:", int(np.argmax(probs_last)))


In [ ]:
d.vocab

In [ ]:
d.label
# En el corazón de la bulliciosa ciudad